In [0]:



from itertools import product
import threading
import psycopg2
from contextlib import closing
import time
import copy

GP_COUNTRY_CODE = [1000, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = []
IOS_DEVICE_ID = [2001]
GRANULARIY = [ "weekly"]
NETWORK = [ 121, 316, 325]
MONTH = ['201904', '201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903']


def select(sql, dbinfo):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


def get_partition_list(granularity=False, device_id=False, store_id=False, network_id=False, month=False):
    granularity_list = GRANULARIY if granularity else [None]
    device_id_list = GP_DEVICE_ID + IOS_DEVICE_ID if device_id else [None]
    store_id_list = GP_COUNTRY_CODE + IOS_COUNTRY_CODE if store_id else [None]
    network_id_list = NETWORK if network_id else [None]
    month_list = MONTH if month else [None]
    product_list = []
    for g, d, s, n, m in product(granularity_list, device_id_list, store_id_list, network_id_list, month_list):
        if (d in GP_DEVICE_ID and s in IOS_COUNTRY_CODE) or (d in IOS_DEVICE_ID and s in GP_COUNTRY_CODE):
            continue
        product_list.append((g,d,s,n, m))
    return product_list


def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str + "\n")


def get_where(table_keys):
    where_condition = []
    if "creative_id" in table_keys:
        where_condition.append("creative_id <= 90100006099406")
    if "kpi" in table_keys:
        where_condition.append( "kpi IS NOT NULL")
    if "update_date" in table_keys:
        where_condition.append("update_date < '2019-05-01'")
    elif "date" in table_keys:
        where_condition.append("date < '2019-05-01'")
    if len(where_condition)==0:
        return "TRUE"
    else:
        return " AND ".join(where_condition)


def _check_mkt_data_from_db(table_name, table_keys, partition, is_distinct=True):
    result = 0
    error_result = 0
    granularity, device_id, store_id, network_id, month = partition
    sql_where = get_where(table_keys)
    sql_table = "{granularity}{table_name}{device_id}{store_id}{network_id}{month}".format(
        table_name=table_name + "_",
        device_id=str(device_id) + "_" if device_id else "",
        store_id=str(store_id) + "_" if store_id else "",
        granularity=str(granularity) + "_" if granularity else "",
        month=str(month) + "_" if month else "",
        network_id=str(network_id) + "_" if network_id else "").rstrip('_')
    # print sql_table
    dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27',
              'PORT': 5433}
    dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437),
              ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]
    for host, port in dblist:  # debug
        dbinfo['HOST'] = host
        dbinfo['PORT'] = port
        try:
            if is_distinct:
                sql_count = "DISTINCT({table_keys})".format(table_keys=table_keys)
            else:
                sql_count = "*"
            sql = "SELECT COUNT({sql_count}) FROM mkt.{sql_table} WHERE {sql_where}".format(
                sql_count=sql_count, sql_table=sql_table, sql_where=sql_where)
            # print sql # debug
            r, _ = select(sql, copy.deepcopy(dbinfo))
            result += r[0][0]
            # print result # debug
        except Exception, e:
            if "does not exist" in e.message:
                error_result = 0
            else:
                print "******error********"
                print e.message
                error_result = -1
    return result or error_result


def _check_mkt_data_from_s3(table_name, table_keys, partition, is_fact):
    result = 0
    error_result = 0
    granularity, device_id, store_id, network_id, month = partition
    s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/{fact}/{granularity}{month}{device_id}{store_id}{network_id}".format(
        table_name=table_name, fact='fact' if is_fact else 'dimension',
        month="month={}/".format(str('-'.join([month[:4], month[-2:]]))) if month else "",
        granularity="granularity={}/".format(granularity) if granularity else "",
        device_id="device_id={}/".format(device_id) if device_id else "",
        store_id="store_id={}/".format(store_id) if store_id else "",
        network_id="network_id={}/".format(network_id) if network_id else "")
    try:
        # print s3_path # debug
        result = spark.read.parquet(s3_path).count()
    except Exception, e:
        # print e.message # debug
        error_result = 0
    return result or error_result


def _check_mkt_data_thread(table_name, table_keys, is_fact):
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(time.time()))
    log_filename = '/tmp/db_{}_{}_{}.log'.format(table_name, timestamp, threading.currentThread().name)
    print log_filename
    while(len(partition_list)>0):
        partitions = partition_list.pop()
        time_start = time.time()
        # SQL RESULT
        sql_result = _check_mkt_data_from_db(table_name, table_keys, partitions, is_distinct=True)
        # S3 RESULT
        s3_result = _check_mkt_data_from_s3(table_name, table_keys, partitions, is_fact)
        duplicated_sql_result = "NA"
        if s3_result > sql_result > 0:
            duplicated_sql_result = _check_mkt_data_from_db(table_name, table_keys, partitions, is_distinct=False)
        time_end = time.time()
        result_str = "{},{},{},{},{},{}".format(table_name, "_".join([str(p) for p in partitions]), sql_result,
                                                duplicated_sql_result, s3_result, int(time_end-time_start))
        write_to_file(log_filename, result_str)

partition_list = []

def check_mkt_data(table_name, table_keys, is_fact, **kw):
    thread_count=10
    thread_group = []
    global partition_list
    partition_list = get_partition_list(**kw) # debug
    print len(partition_list)
    for index in range(0, thread_count):
        t = threading.Thread(
            name="thread-{}".format(index),
            target=_check_mkt_data_thread,
            args=(table_name, table_keys, is_fact))
        thread_group.append(t)
        t.start()

    for t in thread_group:
        t.join()

# debug

check_mkt_data("pub_app_ad_app_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , estimate, kpi",  is_fact=True, granularity=True,device_id=True, store_id=True, network_id=True, month=True )


In [0]:
%%sh

cat /tmp/db_pub_app_ad_app_estimate* | grep -v "0,NA,0"

date
# ls -al /tmp/pub_app_ad_app_detail_estimate_2019-08-25_02-00-33_thread-0.log

# cat /tmp/pub_app_ad_app_detail_estimate_2019-08-25_02-00-33_thread-* | grep -v "0,NA,0"


In [0]:
229840
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-0.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-1.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-2.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-3.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-4.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-5.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-6.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-7.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-8.log
/tmp/pub_app_ad_app_detail_estimate_2019-08-25_07-39-09_thread-9.log
/opt/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [0]:
%%sh



cat /tmp/creative_detail_estimate_2019-08-22_15-26-18.log | grep -v "0,NA,0"


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-pub_app_ad_app_estimate.v1/fact/granularity=monthly/month=2015-04/device_id=1001/store_id=10/network_id=1/

mkt.monthly_pub_app_ad_app_estimate_2002_0_114_201606

# +------------+------------------+-------------+-----------+----------------+---------------+
# | Column     | Type             | Modifiers   | Storage   | Stats target   | Description   |
# |------------+------------------+-------------+-----------+----------------+---------------|
# | device_id  | smallint         |             | plain     | <null>         | <null>        |
# | store_id   | integer          |             | plain     | <null>         | <null>        |
# | network_id | smallint         |             | plain     | <null>         | <null>        |
# | date       | date             |             | plain     | <null>         | <null>        |
# | pub_app_id | bigint           |             | plain     | <null>         | <null>        |
# | ad_app_id  | bigint           |             | plain     | <null>         | <null>        |
# | kpi        | smallint         |             | plain     | <null>         | <null>        |
# | estimate   | double precision |             | plain     | <null>         | <null>        |

In [0]:
%%sh

df -h

In [0]:
%%sh
